<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Data Modeling
</div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
product_df = pd.read_csv('../data/processed/preprocess_QCL.csv')
population_df = pd.read_csv('../data/processed/preprocess_OA.csv')
agri_pop_df = pd.read_csv('../data/processed/preprocess_OEA.csv')
value_df = pd.read_csv('../data/processed/preprocess_QV.csv')

In [3]:
element_df = pd.read_csv('../data/external/element_code.csv')
item_df = pd.read_csv('../data/external/item_code.csv')

In [6]:
product_df.head()

,Domain Code,Area Code,Element Code,Item Code,Year,Unit,Value,Flag,Note
0,QCL,237,5312,711,1961,ha,1000.0,E,NaN
1,QCL,237,5419,711,1961,100 g/ha,7000.0,E,NaN
2,QCL,237,5510,711,1961,t,700.0,E,NaN
3,QCL,237,5312,711,1962,ha,1000.0,E,NaN
4,QCL,237,5419,711,1962,100 g/ha,7000.0,E,NaN


,Domain Code,Area Code,Element Code,Item Code,Year,Unit,Value,Flag,Note
0,QCL,237,5312,711,1961,ha,1000.00,E,NaN
1,QCL,237,5419,711,1961,100 g/ha,7000.00,E,NaN
2,QCL,237,5510,711,1961,t,700.00,E,NaN
3,QCL,237,5312,711,1962,ha,1000.00,E,NaN
4,QCL,237,5419,711,1962,100 g/ha,7000.00,E,NaN
...,...,...,...,...,...,...,...,...,...
15065,QCL,237,5419,1735,2020,100 g/ha,170584.00,E,NaN
15066,QCL,237,5510,1735,2020,t,17006433.11,E,NaN
15067,QCL,237,5312,1735,2021,ha,1002974.00,E,NaN
15068,QCL,237,5419,1735,2021,100 g/ha,171731.00,E,NaN


In [9]:
data = product_df[product_df['Item Code'].isin([56, 27])] # Get data of Rice and Maize
data = product_df[product_df['Item Code'].isin([56, 27])]

,Domain Code,Area Code,Element Code,Item Code,Year,Unit,Value,Flag,Note
4565,QCL,237,5312,56,1961,ha,260200.00,A,NaN
4566,QCL,237,5419,56,1961,100 g/ha,11230.00,E,NaN
4567,QCL,237,5510,56,1961,t,292200.00,A,NaN
4568,QCL,237,5312,56,1962,ha,262020.00,X,Unofficial figure
4569,QCL,237,5419,56,1962,100 g/ha,11987.00,X,Unofficial figure
...,...,...,...,...,...,...,...,...,...
10162,QCL,237,5419,27,2020,100 g/ha,59212.00,A,NaN
10163,QCL,237,5510,27,2020,t,42764999.76,A,NaN
10164,QCL,237,5312,27,2021,ha,7219797.00,A,NaN
10165,QCL,237,5419,27,2021,100 g/ha,60740.00,A,NaN


In [ ]:
data